In [ ]:
"""
Psudocode for bot_evalutaion function:

Way to implement:

def bot_evaluation(bot_signals): 
    # initial values
    cash = 1000
    fee=0.03
    bitcoin = 0.0

    for i in range(len(time_frame)-1):
        close_price=close[i]
        # buy
        if bot_signals[i] == "buy":
            bitcoin =  (cash*(1-fee))/close_price
            cash = 0
        # sell
        elif bot_signals[i] == "sell":
            cash = bitcoin * close_price * (1-fee)
            bitcoin =0
    
    # final evaluation to change back to cash
    close_price=close[-1]
    if bitcoin>0:
        cash = bitcoin * close_price * (1-fee)
        bitcoin =0
    
    return cash
"""

In [ ]:
def bot_evaluation(bot_signals):
    """
    (NEED DEVELOPMENT)
    This function will calculate the total cash earned from the buy/sell signals of the trading bot
    Parameters:
    - bot_signals (list): including the buy/sell and keep signal
    - time_length (int): length of time (could be hourly, daily or yearly)
    Return:
    - cash: the total cash amount earned after the trading time
    """ 
    # initial values
    cash = 1000
    fee=0.03
    bitcoin = 0.0

    for i in range(len(time_length)-1):
        close_price=close[i]
        # buy
        if bot_signals[i] == "buy":
            bitcoin =  (cash*(1-fee))/close_price
            cash = 0
        # sell
        elif bot_signals[i] == "sell":
            cash = bitcoin * close_price * (1-fee)
            bitcoin =0
    
    # final evaluation to change back to cash
    close_price=close[-1]
    if bitcoin>0:
        cash = bitcoin * close_price * (1-fee)
        bitcoin =0
    
    return cash